In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pylab
%matplotlib inline

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
from keras.models import Sequential
from keras.layers import Dense
from inspect import signature
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')
   
    
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
heart_dm= pd.read_csv("../input/heart-disease-uci/heart.csv")
heart_dm.head()

1.Age

2.Sex

3.Chest Pain Type (4 values)

4.Resting Blood Pressure

5.Serum Cholestoral in mg/dl

6.Fasting Blood Sugar > 120 mg/dl

7.Resting Electrocardiographic results (values 0, 1, 2)

8.Maximum Heart Rate Achieved

9.Exercise Induces Angina

10.Oldpeak = ST depression induced by exercise relative to rest

11.The Slope Of The Peak Exercise ST segment

12.Number Of Major Vessels (0-3) Colored By Flourosopy

13.Thal: 3=normal; 6=fixed defect; 7=reversable defect

In [ ]:
# Sex Dictionary
sex_dict = { 0: 'Female',1: 'Male' }

# Chest Pain Dictionary
chest_pain_dict = { 0:'Typical Angina', 1:'Atypical Angina', 2:'Non-Anginal Pain', 3:'Asymptomatic'}

# Blood Sugar  Dictionary
fbs_dict = { 0: 'No Blood Sugar', 1: 'Blood Sugar'}

# Rest ECG
restecg_dict = { 0:'Normal restecg', 1:'ST-T wave abnormality restecg', 2:'ventricular hypertrophy restecg'}

# Exercise Induced Angina
exang_dict = { 0:'Exang No', 1:'Exang Yes'}

# Slope of the peak exercise ST segment 
slope_dict = { 0:'Upsloping', 1:'flat', 2:'Downsloping'}

# Number of Major Vessels (0-3) Colored by Flourosopy
ca_dict = { 0:'Major vessel 0', 1:'Major vessel 1', 2:'Major vessel 2', 3:'Major vessel 3', 4:'Major vessel 4'}

# Thalassemia Dictionary
thal_dict = { 0: 'None', 1: 'Normal', 2:'Fixed Defect',3:'Reversable Defect'}

# Target Dictionary
target_dict = { 0: 'Not Present', 1: 'Present'}

# Creating new DataFrame

In [ ]:
df = heart_dm[['age']]

# trestbps as Resting Blood Pressure
df['Resting Blood Pressure'] = heart_dm['trestbps']

# chol as Serum Cholestoral
df['Serum Cholestoral'] = heart_dm['chol']

# thalach as Max. Heart Rate
df['Max. Heart Rate'] = heart_dm['thalach']

# old peak ST Depression
df['ST Depression'] = heart_dm['oldpeak']

# sex as Sex
df['Sex'] = heart_dm['sex'].apply(lambda x:sex_dict[x])

# thal as Thalassemia
df['Thalassemia'] = heart_dm['thal'].apply(lambda x:thal_dict[x])

# fbs as Thalassemia
df['Fasting Blood Sugar'] = heart_dm['fbs'].apply(lambda x:fbs_dict[x])

# cp as Chest Pain
df['Chest Pain'] = heart_dm['cp'].apply(lambda x:chest_pain_dict[x])

# target as Heart Disease
df['Heart Disease'] = heart_dm['target'].apply(lambda x:target_dict[x])

In [ ]:
df.head()

In [ ]:
sns.set(style = "darkgrid")
sns.countplot(x = "Heart Disease", data = df, palette = "bwr")

In [ ]:
countNoDisease = len(df[df['Heart Disease'] == 'Not Present'])
countHaveDisease = len(df[df['Heart Disease'] == 'Present'])
print("Percentage of Patients Haven't Heart Disease: {:.2f}%".format((countNoDisease / (len(df['Heart Disease'])) * 100)))
print("Percentage of Patients Have Heart Disease: {:.2f}%".format((countHaveDisease / (len(df['Heart Disease'])) * 100)))

In [ ]:


pd.crosstab(df['Sex'], df['Heart Disease']).plot(kind = 'bar', figsize = (13,5), color = ['#0000ff', '#ff0000'])



In [ ]:
pd.crosstab(df['age'], df['Heart Disease']).plot(kind = "bar", figsize = (20,6))

In [ ]:
pd.crosstab(df['ST Depression'], df['Heart Disease']).plot(kind = "bar", figsize = (15,6), color = ['#0000ff','#FF5733' ])

In [ ]:


pd.crosstab(df['Fasting Blood Sugar'], df['Heart Disease']).plot(kind = "bar", figsize = (15,6), color = ['#FFC300','#581845' ])



In [ ]:
pd.crosstab(df['Chest Pain'], df['Heart Disease']).plot(kind = "bar", figsize = (15,6), color = ['#11A5AA','#AA1190' ])

In [ ]:
pd.crosstab(heart_dm.thal, df['Heart Disease']).plot(kind = "bar", figsize = (15,6), color = ['#99A6BB','#AA4510' ])

We plot the heatmap by using the correlation for the dataset. This helps us eliminate any features that may not help with prediction.

In [ ]:
#get correlations of each features in dataset
corrmat = heart_dm.corr()
top_corr_features = corrmat.index
plt.figure(figsize = (20,20))

#plot heat map
sns.heatmap(heart_dm[top_corr_features].corr(), annot = True, cmap = "Blues")

In [ ]:


#Correlation with output variable
cor_target = abs(corrmat["target"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > 0.10]
relevant_features



In [ ]:
X = heart_dm.drop(['fbs','chol','target'], 1)
Y = heart_dm['target']

In [ ]:
#standardizing the input feature

sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [ ]:
def plt_conf_matrix(pred):
  # Confusion Matrix
  cm = confusion_matrix(Y_test, pred)
  plt.figure(figsize = (5, 4))
  sns.heatmap(cm, xticklabels = ['Positive','Negative'], yticklabels = ['Positive','Negative'], annot = True, fmt = 'd', cmap="Blues")
  plt.title('Confusion Matrix')
  plt.ylabel('Actual Values')
  plt.xlabel('Predicted Values')
  plt.show()
  
def plt_var_dev(scores):
  #Variance & Dev. standard
  print('\nVariance: {}'.format(round(np.var(scores) * 100, 2)))
  print('Dev. standard: {}\n'.format(round(np.std(scores) * 100, 2)))
  data = {'variance': np.var(scores), 'standard dev': np.std(scores)}
  names = list(data.keys())
  values = list(data.values())
  fig,axs = plt.subplots(1, 1, figsize = (5, 3), sharey = True)
  axs.bar(names, values)
  plt.show()

def plt_auc(model):
  #AUC
  probs = model.predict_proba(X_test)
  # keep probabilities for the positive outcome only
  probs = probs[:, 1]
  auc = roc_auc_score(Y_test, probs)
  print('\nAUC: %.2f\n' % auc)
  # calculate roc curve
  fpr, tpr, thresholds = roc_curve(Y_test, probs)
  plt.figure(figsize = (5, 4))
  # plot no skill
  plt.plot([0, 1], [0, 1], linestyle = '--')
  # plot the roc curve for the model
  plt.plot(fpr, tpr, marker = '.')
  plt.xlabel('FP RATE')
  plt.ylabel('TP RATE')
  plt.show()
  
def plt_prec_rec(pred):
  #Precision-Recall Curve
  average_precision = average_precision_score(Y_test, pred)
  precision, recall, _ = precision_recall_curve(Y_test, pred)
  print('\nAP = {0:0.2f}\n'.format(average_precision))
  step_kwargs = ({'step': 'post'}
                 if 'step' in signature(plt.fill_between).parameters
                 else {})
  plt.figure(figsize = (5, 4))
  plt.step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
  plt.fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.ylim([0.0, 1.05])
  plt.xlim([0.0, 1.0])
  plt.title('Precision-Recall curve')

# Supervised Algorithms

In [ ]:
scores = []
models_name = []

Stocastich Gradient Descent

In [ ]:
sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
sgd.fit(X_train, Y_train)
y_pred_sgd = sgd.predict(X_test)
score_sgd = round(accuracy_score(y_pred_sgd,Y_test) * 100, 2)

print("The accuracy score achieved using Stochastic Gradient Discent is: " + str(score_sgd) + " %")

scores.append(score_sgd)
models_name.append('SGD')

# Logistic Regression


In [ ]:

lr = LogisticRegression()
lr.fit(X_train,Y_train)
y_pred_lr = lr.predict(X_test)
score_lr = round(accuracy_score(y_pred_lr,Y_test) * 100, 2)

print("The accuracy score achieved using Logistic Regression is: " + str(score_lr) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_lr))

plt_conf_matrix(y_pred_lr)
plt_auc(lr)
plt_prec_rec(y_pred_lr)

scores.append(score_lr)
models_name.append('LGR')

In [ ]:
cv = ShuffleSplit(n_splits = 5, test_size = 0.15, random_state = 0)
scores_lr = cross_val_score(lr, X, heart_dm.target, cv = cv)
score_lr = round(scores_lr.mean() * 100, 2)

print("The accuracy score achieved using LR & Cross-Validation technique is: " + str(score_lr) + " %")

scores.append(score_lr)
models_name.append('LGR')

plt_var_dev(scores_lr)

# K-Nearest Neighbor

In [ ]:
k_range = range(1,10)
error = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train,Y_train)
    y_pred_knn = knn.predict(X_test)
    score_knn = round(accuracy_score(y_pred_knn,Y_test) * 100, 2)
    error.append(np.mean(y_pred_knn != Y_test))
    
plt.figure(figsize=(4,4))
plt.plot(range(1, 10), error)
plt.title('Error Rate K Value')  
plt.xlabel('K Value')  
plt.ylabel('Mean Error')
plt.show()

scores.append(max(scores_list))
models_name.append('KNN')

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(X_train,Y_train)
y_pred_knn = knn.predict(X_test)
score_knn = round(accuracy_score(y_pred_knn, Y_test) * 100, 2)

print("The accuracy score achieved using KNN is: " + str(score_knn) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_knn))

plt_conf_matrix(y_pred_knn)
plt_auc(knn)
plt_prec_rec(y_pred_knn)



In [ ]:
cv = ShuffleSplit(n_splits = 5, test_size = 0.15, random_state = 0)
scores_knn = cross_val_score(knn, X, heart_dm.target, cv = cv)
score_knn = round(scores_knn.mean() * 100, 2)

print("The accuracy score achieved using KNN & Cross-Validation technique is: " + str(score_knn) + " %")

scores.append(score_knn)
models_name.append('KNN')

plt_var_dev(scores_knn)

# Gaussian Naive Bayes - Probabilistic Classifier

In [ ]:


nb = GaussianNB()
nb.fit(X_train,Y_train)
y_pred_nb = nb.predict(X_test)
score_nb = round(accuracy_score(y_pred_nb,Y_test) * 100, 2)

print("The accuracy score achieved using Naive Bayes is: " + str(score_nb) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_nb))

plt_conf_matrix(y_pred_nb)
plt_auc(nb)
plt_prec_rec(y_pred_nb)

scores.append(score_nb)
models_name.append('GNB')

In [ ]:
cv = ShuffleSplit(n_splits = 5, test_size = 0.15, random_state = 0)
scores_nb = cross_val_score(nb, X, heart_dm.target, cv = cv)
score_nb = round(scores_nb.mean() * 100, 2)

print("The accuracy score achieved using Gaussian Naive Bayes & Cross-Validation technique is: " + str(score_nb) + " %")

scores.append(score_nb)
models_name.append('GNB')

plt_var_dev(scores_nb)

# Decision Tree - Probabilistic Classifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
y_pred_dt = dt.predict(X_test)
score_dt = round(accuracy_score(y_pred_dt, Y_test) * 100, 2)

print("The accuracy score achieved using Decision Tree is: " + str(score_dt) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_dt))

scores.append(score_dt)
models_name.append('DT')

plt_conf_matrix(y_pred_dt)
plt_auc(dt)
plt_prec_rec(y_pred_dt)

# Support Vector Machine

In [ ]:


sv = SVC(kernel = 'sigmoid')
sv.fit(X_train, Y_train)
y_pred_svm = sv.predict(X_test)
score_svm = round(accuracy_score(y_pred_svm, Y_test) * 100, 2)

print("The accuracy score achieved using Linear SVM is: " + str(score_svm) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_svm))

plt_conf_matrix(y_pred_svm)
plt_prec_rec(y_pred_svm)

scores.append(score_svm)
models_name.append('SVM')


In [ ]:
cv = ShuffleSplit(n_splits = 5, test_size = 0.15, random_state = 0)
scores_svm = cross_val_score(sv, X, heart_dm.target, cv = cv)
score_svm = round(scores_svm.mean() * 100, 2)

print("The accuracy score achieved using SVM & Cross-Validation technique is: " + str(score_svm) + " %")

scores.append(score_svm)
models_name.append('SVM')

plt_var_dev(scores_svm)

# Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators = 100, bootstrap = True)
rf.fit(X_train, Y_train)
y_pred_rf = rf.predict(X_test)
score_rf = round(accuracy_score(y_pred_rf, Y_test) * 100, 2)

print("The accuracy score achieved using Random Forest is: " + str(score_rf) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred_rf))


scores.append(score_rf)
models_name.append('RF')

plt_conf_matrix(y_pred_rf)
plt_auc(rf)
plt_prec_rec(y_pred_rf)

# Neural Network

In [ ]:
classifier = Sequential()
# First Hidden Layer
classifier.add(Dense(4, activation = 'relu', kernel_initializer = 'random_normal', input_dim = 11))
# Second  Hidden Layer
classifier.add(Dense(4, activation = 'relu', kernel_initializer = 'random_normal'))
# Output Layer
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'random_normal'))

In [ ]:
# Compiling the neural network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Fitting the data to the training dataset
history = classifier.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 100, batch_size = 16, verbose = 2)

In [ ]:
# Model accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()



In [ ]:
# Model Losss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:


y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)



In [ ]:
cm = confusion_matrix(Y_test, y_pred)
true_pos = np.diag(cm)
false_pos = np.sum(cm, axis = 0) - true_pos
false_neg = np.sum(cm, axis = 1) - true_pos
score_nn = round(np.sum(true_pos)/(np.sum(true_pos) + np.sum(false_pos)) * 100, 2)

print("\nThe accuracy score achieved using Neural Network is: " + str(score_nn) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred))


plt_conf_matrix(y_pred)
plt_prec_rec(y_pred)

scores.append(score_nn)
models_name.append('NN')

In [ ]:
true_pos = np.diag(cm)
false_pos = np.sum(cm, axis = 0) - true_pos
false_neg = np.sum(cm, axis = 1) - true_pos
score_nn = round(np.sum(true_pos)/(np.sum(true_pos) + np.sum(false_pos)) * 100, 2)

print("The accuracy score achieved using Neural Network is: " + str(score_nn) + " %")
print ('\nClasification report:\n', classification_report(Y_test, y_pred))

plt_conf_matrix(y_pred)
plt_prec_rec(y_pred)

scores.append(score_nn)
models_name.append('NN')

In [ ]:


print(scores)

colors = ["purple", "green", "orange", "magenta", "red", "yellow", "grey", "blue"]
sns.set_style("whitegrid")
plt.figure(figsize = (8, 5))
plt.yticks(np.arange(0, 100, 10))
plt.ylabel("Accuracy %")
plt.xlabel("Algorithms")
sns.barplot(x = scores, y = models_name, palette = colors)
plt.show()

